In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# upload following zips under /content/ https://drive.google.com/drive/folders/1gT1kZX6Zc6onw9huciYU3Qysl237cavZ?usp=sharing
!cp -r /content/drive/MyDrive/breakout/*.zip /content  
!mkdir $(echo "$(ls data_*.zip)" | cut -f 1 -d '.')
!for zipname in $(ls -d */); do mv "/content/${zipname%%/}.zip" "/content/${zipname}"; cd "/content/${zipname}";unzip -q "/content/${zipname}${zipname%%/}.zip"; done

mv: cannot stat '/content/drive.zip': No such file or directory
unzip:  cannot find or open /content/drive/drive.zip, /content/drive/drive.zip.zip or /content/drive/drive.zip.ZIP.
mv: cannot stat '/content/sample_data.zip': No such file or directory
unzip:  cannot find or open /content/sample_data/sample_data.zip, /content/sample_data/sample_data.zip.zip or /content/sample_data/sample_data.zip.ZIP.


In [ ]:
import os
import plotly.express as px
import numpy as np
import json
import imageio


In [ ]:
shifted_metadata_list = metadata_list[:-6]

In [ ]:
len(metadata_list)

3506

In [ ]:
# there is an issue with data_4 since collides with data_3 because is shifted -1 position. Same thing happens with data_5 and data_6. (-3)
metadata_list = []
for dataset in ["data_1", "data_2", "data_3", "data_4", "data_5", "data_6","data_7","data_8","data_9"]:
  policy_path = f"/content/{dataset}/policy"
  policies = os.listdir(policy_path)
  policies.sort(key=lambda filename: int(filename.split(".")[0]))
  for policy in policies:
    with open(f"{policy_path}/{policy}", "r") as data:
      metadata = json.loads(data.read())["metadata"]
      metadata_list.append(metadata)

iterations = list(range(len(metadata_list)))
shifted_metadata_list = metadata_list[:-5]
print(metadata_list)

[{'policy_name': 'policy_1.json', 'policy_number': 1, 'played_iterations': 115, 'score': 1, 'number_of_new_states': 30, 'number_of_total_states': 30}, {'policy_name': 'policy_2.json', 'policy_number': 2, 'played_iterations': 55, 'score': 0, 'number_of_new_states': 12, 'number_of_total_states': 42}, {'policy_name': 'policy_3.json', 'policy_number': 3, 'played_iterations': 53, 'score': 0, 'number_of_new_states': 5, 'number_of_total_states': 47}, {'policy_name': 'policy_4.json', 'policy_number': 4, 'played_iterations': 161, 'score': 3, 'number_of_new_states': 24, 'number_of_total_states': 71}, {'policy_name': 'policy_5.json', 'policy_number': 5, 'played_iterations': 122, 'score': 2, 'number_of_new_states': 3, 'number_of_total_states': 74}, {'policy_name': 'policy_6.json', 'policy_number': 6, 'played_iterations': 162, 'score': 3, 'number_of_new_states': 13, 'number_of_total_states': 87}, {'policy_name': 'policy_7.json', 'policy_number': 7, 'played_iterations': 52, 'score': 0, 'number_of_ne

# Score Analysis

In [ ]:
fig = px.line(x=iterations, y=[meta["score"] for meta in metadata_list], labels={'x':'iteration', 'y':'score'})
fig.show()

In [ ]:
# Divide meta in sets of 100 iterations representing the x-axis. The y-axis will be the total number of unique scores reached in the iterations set.
# only data_1,2,3,4
step = 100
unique_scores = set([meta["score"] for meta in shifted_metadata_list]) # give a shit about performance rn. so fuck off
data = {
    "set_range": {},
    "score": {},
    "count": {}
}
index = 0
for i in range(0, len(shifted_metadata_list), step):
  set_name = f"[{i+1}, {i+step+1}]"
  for unique_score in unique_scores:
    data["set_range"][index] = set_name
    data["score"][index] = str(unique_score)
    data["count"][index] = sum([1 for meta in shifted_metadata_list[i:i+step+1] if meta["score"] == unique_score])
    index += 1

fig = px.bar(data, x="set_range", y="count", color="score", title="Unique Score Counter. First 1000 iterations")
fig.show()

In [ ]:
# plot maximus score points picks hitted; TODO
data = {
    "iteration": {},
    "max_score": {}
}
max_score = -10e3
index = 0
for i, meta in enumerate(metadata_list):
  if max_score <= meta["score"]:
    max_score = meta["score"]
    data["iteration"][index] = i
    data["max_score"][index] = meta["score"]
    index+=1

fig = px.line(data, x="iteration", y="max_score", markers=True)
fig.show()

In [ ]:
# plot avg score in sets of 100
step = 100
data = {
    "set_range": {},
    "avg_score": {},
}
for e, i in enumerate(list(range(0, len(shifted_metadata_list), step))):
  set_name = f"[{i+1}, {i+step+1}]"
  data["set_range"][e] = set_name
  data["avg_score"][e] = sum([meta["score"] for meta in shifted_metadata_list[i:i+step+1]]) / step

fig = px.line(data, x="set_range", y="avg_score", markers=True)
fig.show()

 # Played frames analysis

In [ ]:
fig = px.line(x=iterations, y=[meta["played_iterations"] for meta in metadata_list], labels={'x':'iteration', 'y':'played frames'})
fig.show()

In [ ]:
import numpy as np
y1 = [meta["score"] for meta in metadata_list]
y2 = [meta["played_iterations"] for meta in metadata_list]
np.corrcoef(y1, y2)

array([[1.        , 0.97504001],
       [0.97504001, 1.        ]])

In [ ]:
fig = px.line(x=iterations, y=[meta["number_of_new_states"] for meta in metadata_list], labels={'x':'iteration', 'y':'number of new states'})
fig.show()

In [ ]:
step = 50
data = {
    "set_range": {},
    "avg_discovered_states": {},
}
for e, i in enumerate(list(range(0, len(shifted_metadata_list), step))):
  set_name = f"[{i+1}, {i+step+1}]"
  data["set_range"][e] = set_name
  data["avg_discovered_states"][e] = sum([meta["number_of_new_states"] for meta in shifted_metadata_list[i:i+step+1]]) / step

fig = px.line(data, x="set_range", y="avg_discovered_states")
fig.show()

In [ ]:
fig = px.line(x=iterations, y=[meta["number_of_total_states"] for meta in metadata_list], labels={'x':'iteration', 'y':'number of total states'})
fig.show()

# Make Gif

In [ ]:
! ffmpeg -i "/content/data_5/agent_run/477/%d.png" /content/data_5_agent_run_1477.gif
! ffmpeg -i "/content/data_5/agent_run/480/%d.png" /content/data_5_agent_run_1480.gif

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li